In [ ]:
import random
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from prettytable import PrettyTable
import sklearn
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit, train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Loading the merged train and test dataset

train_data = pd.read_pickle('merged_train.pkl')
test_data = pd.read_pickle('merged_test.pkl')

In [ ]:
print("*"*45)
print("\n Train Data Shape : {} \n".format(train_data.shape))
print("\n Test Data Shape : {} \n".format(test_data.shape))
print("*"*45)

*********************************************

 Train Data Shape : (590540, 434) 


 Test Data Shape : (506691, 433) 

*********************************************


## Utility Functions
<br>

In [ ]:
def cat_num_features(df):
    
    '''
        Utility Function to get the names of Categorical Features and 
        Numerical Features of the given Dataset.
    '''
    
    catf = []
    numf = []
    
    # Given Categorical Features 
    catf = ['ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', \
            'card6', 'addr1', 'addr2', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', \
            'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', \
            'DeviceType', 'DeviceInfo']
    catf+=['id_'+str(i) for i in range(12,39)]


    # Updating the Categorical Feature Names List based on the columns present in the dataframe
    catf = [feature for feature in catf if feature in df.columns.values]
    numf = [feature for feature in df.columns if feature not in catf and not feature == 'isFraud']
    
    return (catf, numf)  

In [ ]:
def label_encode(X_train, X_test, catf):
  
  '''
    Utility Function to Encode Categorical Features.
  '''

  for f in catf:
    
    X_train[f] = X_train[f].astype(str)
    X_test[f] = X_test[f].astype(str)
    
    le = LabelEncoder()
    le.fit(X_train[f])
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    X_train[f] = le.transform(X_train[f])
    
    # Manually Encoding the CV and Test Dataset so as to avoid error for any category which is not present in train set
    
    # All the categories which are not present in train datset are encoded as -1
    
    X_test[f] = [-1 if mapping.get(v, -1)==-1 else mapping[v] for v in X_test[f].values ]

  return (X_train, X_test)

In [ ]:
def normalize(X_train, X_test):
    '''
        Utility Function to scale the values of the Train, CV and Test Datasets between 0 and 1.
    '''
    
    for f in X_train.columns:

        min_val = X_train[f].min()
        max_val = X_train[f].max()
        
        X_train[f] = (X_train[f]-min_val)/(max_val-min_val)
        X_test[f] = (X_test[f]-min_val)/(max_val-min_val)
        
    return (X_train, X_test)

In [ ]:
def predict_and_save(prediction, name):
    
    '''
        Utility Function to save the test data predictions locally.
    '''

    df = pd.DataFrame({'TransactionID':test_ids.reshape(-1), 'isFraud':prediction.reshape(-1)})
    df = df.sort_values('TransactionID')
    df.to_csv(name, index=False)

## Data Preparation
<br>

### Splitting the Dataset
<br>

In [ ]:
X_train = train_data.drop(['isFraud', 'TransactionID'], axis=1)
y_train = train_data['isFraud']

X_test = test_data.drop(['TransactionID'], axis=1)
test_ids = test_data['TransactionID'].values

del train_data, test_data

In [ ]:
print("*"*45)
print("\n Train Data Shape : {} \n".format(X_train.shape))
print("\n Test Data Shape : {} \n".format(X_test.shape))
print("*"*45)

*********************************************

 Train Data Shape : (590540, 432) 


 Test Data Shape : (506691, 432) 

*********************************************


In [ ]:
# Storing Categorical and Numerical Feature Names 

catf, numf = cat_num_features(X_train)
categorical_feature_indices = [X_train.columns.get_loc(f) for f in catf]

#### Label Encoding Categorical Features
<br>
We will be creating three sets, one having imputed missing values to be used with models which do not handle missing values on their own and the other whose missing values will be imputed and also it will be scaled between 0 and 1 to be used with models like Naive Bayes and Logisitic Regression and the third which is to be used with model like XgBoost which can handle missing values on its own.
<br><br>

In [ ]:
# Imputing the missing values of Categorical Columns with "missing"

X_train[catf] = X_train[catf].fillna('missing')
X_test[catf] = X_test[catf].fillna('missing')


# Label Encoding Categorical Features

X_train, X_test = label_encode(X_train, X_test, catf)

In [ ]:
# Set1 (Imputed and Normalized)

X_train1 = X_train.fillna(-999)
X_test1 = X_test.fillna(-999)
X_train1, X_test1 = normalize(X_train1, X_test1)



# Set2 (Imputed)

X_train2 = X_train.fillna(-999)
X_test2 = X_test.fillna(-999)



# Set3 (Raw)

X_train3 = X_train
X_test3 = X_test

del X_train, X_test

In [ ]:
train1 = X_train1
train1['isFraud'] = y_train

test1 = X_test1

del X_train1, X_test1



train2 = X_train2
train2['isFraud'] = y_train

test2 = X_test2

del X_train2, X_test2



train3 = X_train3
train3['isFraud'] = y_train

test3 = X_test3

del X_train3, y_train, X_test3

In [ ]:
y_train = train1.pop('isFraud')
_ = train2.pop('isFraud')
_ = train3.pop('isFraud')


X_train1 = train1
X_test1 = test1 

X_train2 = train2
X_test2 = test2 

X_train3 = train3
X_test3 = test3 

del train1, train2, train3

In [ ]:
print("*"*60)
print("\n Train Dataset Set1 Shape : {} \n".format(X_train1.shape))
print("\n Test Dataset Set1 Shape : {} \n".format(X_test1.shape))
print("*"*60)
print("\n Train Dataset Set1 Shape : {} \n".format(X_train1.shape))
print("\n Test Dataset Set2 Shape : {} \n".format(X_test2.shape))
print("*"*60)
print("\n Train Dataset Set1 Shape : {} \n".format(X_train1.shape))
print("\n Test Dataset Set3 Shape : {} \n".format(X_test3.shape))
print("*"*60)

************************************************************

 Train Dataset Set1 Shape : (590540, 432) 


 Test Dataset Set1 Shape : (506691, 432) 

************************************************************

 Train Dataset Set1 Shape : (590540, 432) 


 Test Dataset Set2 Shape : (506691, 432) 

************************************************************

 Train Dataset Set1 Shape : (590540, 432) 


 Test Dataset Set3 Shape : (506691, 432) 

************************************************************


## Base Line Models
<br>

<br>

### 1. Naive Bayes

<br>

In [ ]:
nb = GaussianNB(priors=[0.5,0.5]) 
nb.fit(X_train1, y_train)

GaussianNB(priors=[0.5, 0.5])

In [ ]:
nb_test_proba = nb.predict_proba(X_test1)[:,1]

In [ ]:
predict_and_save(nb_test_proba, 'nb_pred_bl.csv')

> __Kaggle Submission__

<img src= https://i.ibb.co/Jv1f53w/Screenshot-from-2021-06-08-17-20-04.png>

<br>

### 2. Logistic Regression

<br>

In [ ]:
lr = LogisticRegression(n_jobs = -1, class_weight = 'balanced', random_state = 3) 
lr.fit(X_train1, y_train)

LogisticRegression(class_weight='balanced', n_jobs=-1, random_state=3)

In [ ]:
lr_test_proba = lr.predict_proba(X_test1)[:,1]

In [ ]:
predict_and_save(lr_test_proba, 'lr_pred_bl.csv')

> __Kaggle Submission__

<img src= https://i.ibb.co/FqGD60V/Screenshot-from-2021-06-08-17-20-56.png>

<br>

### 3. Decision Tree

<br>

In [ ]:
dt =  DecisionTreeClassifier(random_state=3, class_weight='balanced')
dt.fit(X_train2, y_train)

DecisionTreeClassifier(class_weight='balanced', random_state=3)

In [ ]:
dt_test_proba = dt.predict_proba(X_test2)[:,1]

In [ ]:
predict_and_save(dt_test_proba, 'dt_pred_bl.csv')

> __Kaggle Submission__

<img src= https://i.ibb.co/dkJ67ym/Screenshot-from-2021-06-08-17-21-47.png>

<br>

### 4. Random Forest

<br>

In [ ]:
rf =  RandomForestClassifier(n_jobs = -1, class_weight = 'balanced', random_state = 3)
rf.fit(X_train2, y_train)

RandomForestClassifier(class_weight='balanced', n_jobs=-1, random_state=3)

In [ ]:
rf_test_proba = rf.predict_proba(X_test2)[:,1]

In [ ]:
predict_and_save(rf_test_proba, 'rf_pred_bl.csv')

> __Kaggle Submission__

<img src= https://i.ibb.co/bNBXMPm/Screenshot-from-2021-06-08-17-22-29.png>

<br>

### 5. Adaptive Boosting

<br>

In [ ]:
ab =  AdaBoostClassifier(random_state = 3)
ab.fit(X_train2, y_train)

AdaBoostClassifier(random_state=3)

In [ ]:
ab_test_proba = ab.predict_proba(X_test2)[:,1]

In [ ]:
predict_and_save(ab_test_proba, 'ab_pred_bl.csv')

> __Kaggle Submission__

<img src= https://i.ibb.co/2SJZgjv/Screenshot-from-2021-06-08-17-27-05.png>

<br>

### 6. Gradient Boosted Decision Tree (XGBoost)

<br>

In [ ]:
weight = y_train.value_counts()[0]/y_train.value_counts()[1]

In [ ]:
xgboost =  XGBClassifier(scale_pos_weight = weight, objective='binary:logistic', eval_metric = 'auc', random_state = 3, tree_method = 'gpu_hist')
xgboost.fit(X_train3, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=3,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=27.579586700866283,
              seed=None, silent=None, subsample=1, tree_method='gpu_hist',
              verbosity=1)

In [ ]:
xgboost_test_proba = xgboost.predict_proba(X_test3)[:,1]

In [ ]:
predict_and_save(xgboost_test_proba, 'xgboost_pred_bl.csv')

__Kaggle Submission__

<img src= https://i.ibb.co/TRB96hh/Screenshot-from-2021-06-08-17-47-20.png>

## Conclusion

From the above results, it is quite clear that the Decision Tree based Ensemble is working best for our problem. Hence, from now on we will be using Tree based Ensemble only and will try to optimize the data and models to get more better score.